In [15]:
import pandas as pd
import numpy as np
import warnings

data = pd.read_csv('../data/ERP_data.csv')
labels = list(data.columns.values)
del labels[0]
del labels[0]

targets = data['Phenotype']
del data['Subject']
del data['Phenotype']

In [16]:
from sklearn import preprocessing
from sklearn import feature_selection
from sklearn import cross_validation

folds = 10

imp = preprocessing.Imputer()
data = imp.fit_transform(data, targets)
data = preprocessing.scale(data)
anova_selection_logreg = feature_selection.SelectKBest(feature_selection.f_classif, k=21)
anova_selection_svm = feature_selection.SelectKBest(feature_selection.f_classif, k=16)
anova_selection_neighbors = feature_selection.SelectKBest(feature_selection.f_classif, k=22)
anova_selection_rf = feature_selection.SelectKBest(feature_selection.f_classif, k=11)
anova_selection_gboost = feature_selection.SelectKBest(feature_selection.f_classif, k=12)

In [17]:
from sklearn import linear_model, svm, neighbors, ensemble
from sklearn import pipeline
from sklearn import metrics

logreg_model = linear_model.LogisticRegression(C=.001)
svm_model = svm.SVC(C=0.66)
neighbors_model = neighbors.KNeighborsClassifier(n_neighbors=13)
rf_model = ensemble.RandomForestClassifier(n_estimators=20, min_samples_split=3, min_samples_leaf=2)
gboost_model = ensemble.GradientBoostingClassifier(min_samples_split=35, learning_rate=0.5, n_estimators=110)
stack_logreg_model = linear_model.LogisticRegression(C=1)
'''
logreg_pipe = pipeline.make_pipeline(anova_selection_logreg, logreg_model)
svm_pipe = pipeline.make_pipeline(anova_selection_svm, svm_model)
neighbors_pipe = pipeline.make_pipeline(anova_selection_neighbors, neighbors_model)
rf_pipe = pipeline.make_pipeline(anova_selection_rf, rf_model)
gboost_pipe = pipeline.make_pipeline(anova_selection_gboost, gboost_model)
'''
pipes = []
pipes.append(pipeline.make_pipeline(anova_selection_logreg, logreg_model))
pipes.append(pipeline.make_pipeline(anova_selection_svm, svm_model))
pipes.append(pipeline.make_pipeline(anova_selection_neighbors, neighbors_model))
pipes.append(pipeline.make_pipeline(anova_selection_rf, rf_model))
pipes.append(pipeline.make_pipeline(anova_selection_gboost, gboost_model))

accuracies = []
precisions = []
recalls = []
fscores = []
confusions = []
warnings.filterwarnings("ignore")
for i in range(1):
    cross_val = cross_validation.StratifiedKFold(targets, n_folds=folds, shuffle=True)
    y_truth = []
    y_preds = []
    for train, test in cross_val:
        X_train, X_test, y_train, y_test = data[train], data[test], targets[train], targets[test]
        pred_features = np.transpose([clf.fit(X_train, y_train).predict(data) for clf in pipes])
        y_truth.append(y_test)
        y_preds.append(stack_logreg_model.fit(pred_features[train], y_train).predict(pred_features[test]))
    
    accuracy = metrics.accuracy_score(y_truth, preds)
    precision, recall, fscore, support = metrics.precision_recall_fscore_support(
        y_truth, preds, average='binary', pos_label='AD')
    confusion = metrics.confusion_matrix(y_truth, preds, labels=['HC', 'AD'])
    accuracies.append(accuracy)
    precisions.append(precision)
    recalls.append(recall)
    fscores.append(fscore)
    confusions.append(confusion)

print("Accuracy: {0}, with std: {1}".format(np.mean(accuracies), np.std(accuracies)))
print("Precision: {0}".format(np.mean(precisions)))
print("Recall: {0}".format(np.mean(recalls)))
print("F1 Score: {0}".format(np.mean(fscores)))
print("Confusion Matrix:\n   HC     AD\n{0}".format(np.mean(confusions, axis=0)))

ValueError: could not convert string to float: 'AD'